In [1]:
#Import libraries and connect to DWH
import pandas as pd
import psycopg2
import time
from customer_io import Customer_io

#Conectarse al DWH no olviden poner su usuario y contraseña de redshift
conn = psycopg2.connect(
                        host='live-main.c56d9jndqmv2.us-east-2.redshift.amazonaws.com',
                        port=5439,
                        database='klarprod',
                        user='tf_jose_gr',
                        password='HAjZbUzk&'
                       )

#Definimos query, recordar que columna de email tiene que llamarse "email", y todas las demás columnas serán atributos
query = """with salary_advance_data as (
select user_id, days_past_due
from credit.salary_advance_loanbook
where loan_status = 'DELINQUENT'
)
select  kyc.email, sa.days_past_due as adelanto_days_past_due
from salary_advance_data sa
left join klar_pii.db_kyc_public_user_kyc kyc on sa.user_id=kyc.id
where adelanto_days_past_due<=7"""

#Línea para guardar el query en un dataframe df
df = pd.read_sql(query, conn)

#Cambiar formato para números enteros en texto (string)
df['adelanto_days_past_due'] = df['adelanto_days_past_due'].map(str)

print(df)

#Cerrar conexión porque están limitadas (máx dependiendo de lo que asigna data)
conn.close()

segment = 1512
attributes = ['adelanto_days_past_due']

/var/folders/b_/lql2bh7s2797fh2q6lw2ttdh0000gn/T/ipykernel_7447/9411911.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


                               email adelanto_days_past_due
0          erickestrella92@gmail.com                      4
1               karmona776@gmail.com                      4
2   iluminaciontecnica01@hotmail.com                      4
3                jc_meji@hotmail.com                      3
4               minegb1111@gmail.com                      3
5          yahircruzsolis8@gmail.com                      4
6           ruchyramirez73@gmail.com                      7
7                egfairy10@gmail.com                      7
8              pajim2909@outlook.com                      4
9              andii.c.c39@gmail.com                      7
10         julioarvizu1234@gmail.com                      4
11           fhptotalguard@gmail.com                      6
12            irislomeli@hotmail.com                      3
13       c.martinez.huawei@gmail.com                      7
14  danielrodriguezhorta14@gmail.com                      5
15            kiara_1985@hotmail.com    

In [2]:
cio = Customer_io()
if len(df) > 0:
    # Get old users from the segment
    old_users = cio.get_users_segment(segment)
    # -----------------------------------------------------------------------------------------
    # If old users greater than 0 then delete users from segment
    if len(old_users) > 0:
        # Delete users from segment
        # If the users from segment is greater than 1000
        # Additional logic is need to delete those users by 1000 users chunks
        chunk_size = len(old_users)
        # Get the integer for the division
        i = (chunk_size // 1000) + 1
        # For loop to iterate over the list
        for idx in range(i):
            # Generate the list of users
            aux_old_users = old_users[idx*1000:(idx+1)*1000]
            # Delete the users
            cio.del_users_segment(segment, aux_old_users)
            # Wait 30 seconds
            time.sleep(30)
    # ----------------------------------------------------------------------------------------
    # Add Users to the segment
    # Sleep 70 second to avoid conflicts with campaigns
    time.sleep(70)
    # Add the users to the segment
    i = (df.shape[0] // 1000) + 1
    # For loop to iterate over the list
    for idx in range(i):
        # Generate the list of users
        aux_new_users = df[idx*1000:(idx+1)*1000]
        # Delete the users
        cio.add_users_segment(segment, aux_new_users, attributes)
        # Wait 30 seconds
        time.sleep(30)
    # ----------------------------------------------------------------------------------------
    # Sleep time
    time.sleep(4)
    # Get the users to see if all users enter correctly to the segment
    new_users = cio.get_users_segment(segment)
    # Print the lent of users
    print(len(new_users))
    # Check if all users are in the segment
    if len(new_users) != len(df):
        print('Not all users are in the campaign, please verify')
    else:
        print("No campaign")

/Users/gabrielreynoso/PycharmProjects/klar/Business_Logic_Validation/CRM/customer_io.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['md5em'] = df.email.apply(lambda x: (hashlib.md5(x.encode())).hexdigest())


erickestrella92@gmail.com
b9780b83088688ae15f38a9931491420
{'adelanto_days_past_due': '4'}
karmona776@gmail.com
ead3d714b777dd4cfd1f837da1ed0bce
{'adelanto_days_past_due': '4'}
iluminaciontecnica01@hotmail.com
4b839345fe5ee383e664a99dde5395b0
{'adelanto_days_past_due': '4'}
jc_meji@hotmail.com
78d31cb492b773d2fa968bba716f5bbd
{'adelanto_days_past_due': '3'}
minegb1111@gmail.com
cf60c1b2da4343ffb36b9510139cfd26
{'adelanto_days_past_due': '3'}
yahircruzsolis8@gmail.com
b5a923f044b50d661730158f620a7b23
{'adelanto_days_past_due': '4'}
ruchyramirez73@gmail.com
9e518783eec4660d89f304ac5115a9b3
{'adelanto_days_past_due': '7'}
egfairy10@gmail.com
a0fd94f1c9fb4ce8506c989c4de99359
{'adelanto_days_past_due': '7'}
pajim2909@outlook.com
0d7497acb3d309dbb029c74288a1b251
{'adelanto_days_past_due': '4'}
andii.c.c39@gmail.com
cc88f9414bf8134886f80daac2d723a9
{'adelanto_days_past_due': '7'}
julioarvizu1234@gmail.com
54b24f500dab1a3fbdffe0d51c1c6b00
{'adelanto_days_past_due': '4'}
fhptotalguard@gmail.com